In [1]:
import csv
import json
import random
import re
from collections import Counter, defaultdict

import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import numpy as np
import pandas as pd
import seaborn as sns

import torch

from datasets import (
    Dataset,
    DatasetDict,
    concatenate_datasets,
    load_dataset,
    load_from_disk,
)
from transformers import (
    AutoModelForTokenClassification,
    AutoTokenizer,
    DataCollatorForTokenClassification,
    Trainer,
    TrainingArguments,
    set_seed,
)

import evaluate

from nameparser import HumanName
from names_dataset import NameDataset, NameWrapper
from ethnicseer import EthnicClassifier
import nltk
from nltk.corpus import wordnet as wn

import pycountry_convert as pc
import pycountry
import pickle

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report, cohen_kappa_score

from transformers import BertTokenizerFast, BertForTokenClassification
from datasets import ClassLabel
from evaluate import load as load_metric
from sklearn.metrics import cohen_kappa_score
from itertools import combinations
import krippendorff

In [2]:
conll_main = load_from_disk("../splits/conll_main")

ontonotes_main = load_from_disk("../splits/ontonotes_main")

In [8]:
df = pd.read_excel(
    "../Guided-Adversarial-Augmentation-main/Guided-Adversarial-Augmentation-main/data/data/conll2003/challenge_set.xlsx", header=None)

In [9]:
examples = []

i = 0
while i < len(df):
    row = df.iloc[i]
    if str(row[0]).startswith("GUID"):

        guid = str(df.iloc[i][1]).strip()

        try:
            quality = int(str(df.iloc[i+1][1]).strip())
        except (ValueError, TypeError):
            quality = 999

        try:
            aug_type = int(str(df.iloc[i+2][1]).strip())
        except (ValueError, TypeError):
            aug_type = 999

        tokens_row = df.iloc[i+3].dropna().tolist()[1:]
        labels_row = df.iloc[i+4].dropna().tolist()[1:]
        labels_row = [label.strip()
                      for label in labels_row if label.strip() != ""]

        if len(tokens_row) == len(labels_row):
            examples.append({
                "guid": guid,
                "quality": quality,
                "aug_type": aug_type,
                "tokens": tokens_row,
                "labels": labels_row
            })

        i += 6
    else:
        i += 1

challenge_dataset = Dataset.from_list(examples)

In [31]:
# Label mapping
conll_label_to_id = {
    'O': 0,
    'B-PER': 1, 'I-PER': 2,
    'B-ORG': 3, 'I-ORG': 4,
    'B-LOC': 5, 'I-LOC': 6,
    'B-MISC': 7, 'I-MISC': 8,
}

# Reverse the dict: id → label
id_to_conll_label = {v: k for k, v in conll_label_to_id.items()}

In [32]:
from pathlib import Path
import pickle
from datasets import Dataset, Features, Sequence, Value

for model_name in ['bert-base-cased', 'bert-base-uncased']:
    for train_data_name in ['conll', 'onto']:
        pred_file = Path(f'./results/{model_name}_{train_data_name}_stanford')
        with open(pred_file, 'rb') as f:
            stanford_predictions, _, _ = pickle.load(f)

        stanford_pred_labels = [
            [id_to_conll_label[np.argmax(token_logits)]
             for token_logits in sentence_logits]
            for sentence_logits in stanford_predictions
        ]

        # Validate length
        assert len(stanford_pred_labels) == len(challenge_dataset), \
            f"Prediction length mismatch for {
                model_name} trained on {train_data_name}"

        # Add predictions using .map to avoid Arrow 1D error
        column_name = f"{model_name}_{train_data_name}_predictions"
        challenge_dataset = challenge_dataset.map(
            lambda example, idx: {column_name: stanford_pred_labels[idx]},
            with_indices=True
        )

Map:   0%|          | 0/1418 [00:00<?, ? examples/s]

Map:   0%|          | 0/1418 [00:00<?, ? examples/s]

Map:   0%|          | 0/1418 [00:00<?, ? examples/s]

Map:   0%|          | 0/1418 [00:00<?, ? examples/s]